In [294]:
from random import randint, shuffle, random
import numpy as np

# Matrice d'adjacence

In [295]:
# Génération de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(2, 10)
            if i == j:
                cout = 0
            k = i-1 if i-1 > 0 else n_ville
            if (i+1) % n_ville == j or k == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout

    return mat
 

Initialisation de la population

In [296]:
def population_init(nb_ville, size):
    population = []
    
    for _ in range(size):
        circuit = list(range(nb_ville))
        shuffle(circuit) # circuit random
        population.append([circuit, 5])
        
    return population



In [297]:

def empty_mat(width):
    return [[0]*width for _ in range(width)]

class Agent:
    def __init__(self, nodes_to_explore, deposit_Q, prob_explore_Y):
        self.starting_node = 0 # Toute les fourmies commencent au même endroit
        self.deposit_Q = deposit_Q
        self.null_prob = prob_explore_Y 
        self.reset_exploration(list(nodes_to_explore)) 
    
     
    def reset_exploration(self, nodes_to_explore):
        self.unexplored = nodes_to_explore
        self.unexplored.remove(self.starting_node)
        self.explored = [self.starting_node]
    
    def get_travel_length(self, mat_cout):
        travel_length = 0
        for i in range(1, len(self.explored)):
            depart = self.explored[i-1]
            arrival = self.explored[i]
            
            travel_length += mat_cout[depart][arrival]
            
        return travel_length
    
    def deposite_pheromone(self, mat_cost, pheromone_mat):
        
        pheromone_quantity = self.deposit_Q /  self.get_travel_length(mat_cost)
        
        
        for i in range(1, len(self.explored)):
            depart = self.explored[i-1]
            arrival = self.explored[i]
            
            # On dépose les phéromones sur les arrettes
            pheromone_mat[depart][arrival] += pheromone_quantity
          
          
      
    def choose_next_location(self, pheromone_mat):
        # On choisit la prochaine ville a explorer aléatoirement selon les phéromones,
        # avec une proba non nul qu'on explore une ville jamais exploré par les agents
        # Et retourne un indice parmis la liste des unexplored
        
        depart = self.explored[-1] 
        sum_pheromone = 0
        
        # On somme les differente pheromone pour avoir les probabilité
        for arrival in self.unexplored:
            if pheromone_mat[depart][arrival] > 0:
                sum_pheromone += pheromone_mat[depart][arrival]
            else:
                # On a une proba non nul pour qu'un agent puisse explorer une ville jamais exploré par les autres agents
                sum_pheromone += self.null_prob
        
        # random qui decide la prochaine ville
        r = random() *  sum_pheromone
        s = 0 
        i = 0
        
        # Recherche de la ville qui vient d'etre choisit
        for arrival in self.unexplored: 
            if pheromone_mat[depart][arrival] > 0:
                s += pheromone_mat[depart][arrival]
            else:
                s += self.null_prob
            
            # On s'arrette lorsque l'on a trouvé la prochaine ville, et on retourne son indice dans unexplored
            if s >= r:
                return i
            i += 1
        
        # Flottant
        return i

    
    def advance(self, mat_cost, pheromone_mat): # avance l'agent
        
        # Si on a atteint toutes les villes 
        if len(self.unexplored) <= 0:
            # Une fois la tournée terminé, l'agent dépose les phéromones
            self.deposite_pheromone(mat_cost, pheromone_mat)
            # On reset les villes explorer et on recommence l'exploration 
            self.reset_exploration(self.explored)
         
        # On choisit la prochaine destination parmis ceux qui reste à explorer 
        next_i = self.choose_next_location(pheromone_mat) 
        curr_node = self.unexplored.pop(next_i)
        self.explored.append(curr_node)
        
        
def evaporation(pheromone_mat, disipation_rate):
    for i in range(len(pheromone_mat)):
        for j in range(len(pheromone_mat[0])):
            if pheromone_mat[i][j] > 0:
                pheromone_mat[i][j] -= disipation_rate
                
            

In [298]:
def index_max(arr, exclude_indices): # retourne l'indice dont la valeur est maximal, et qui n'est pas dans exclude_indices
    # On n'explore pas a partir du premier, au cas l'element 0 fait partir de exclude_indices
    best_i = None 
    for i in range(len(arr)): 
        if best_i == None:
            if i not in exclude_indices: 
                best_i = i
            continue 
        
        if arr[best_i] < arr[i] and i not in exclude_indices:
            best_i = i
            
    return best_i

def find_best(pheromone_mat): # Trouve le chemin le plus parcourus
    res = [0]
    i = 1
    
    while i < len(pheromone_mat):
        
        depart = res[-1]
        arrival = index_max(pheromone_mat[depart], res)
        res.append(arrival)
        
        i += 1
        
    return res

def cost(cost_mat, path):
    total_cost = 0
    
    for i in range(1, len(path)):
        depart = path[i - 1]
        arrival = path[i]
        total_cost += cost_mat[depart][arrival]
    
    return total_cost  

def main():
    size = 5
    epoch_max = 1000
    agents_number = 10 
    disipation_rate = 1
    deposit_Q = 1
    prob_explore_Y = 1
    
    mat_cost = create_matrix(size)
    pheromone_mat = empty_mat(size)
    agents = [Agent(list(range(size)), deposit_Q, prob_explore_Y) for _ in range(agents_number)]
    
    disipation_rate = 0
    
    for i in range(epoch_max): 
        
        #print("iteration: ", i)
        # On avance chaque agents dans leurs exploration
        for agent in agents:
            agent.advance(mat_cost, pheromone_mat)

        evaporation(pheromone_mat, disipation_rate)
    
    best_path = find_best(pheromone_mat) 
    print("Meilleur chemin: ", best_path)
    print("Cout: ", cost(mat_cost, best_path))
    
main()

Meilleur chemin:  [0, 4, 6, 5, 1, 2, 3]
Cout:  13.0
